# Read Normally distributed scenarios an makes them ready for training LDF

In [ ]:
import numpy as np
from opendssdirect import dss
import os
import json
import logging

class LoadScenarioGeneration:
    def __init__(self, dss_file, output_dir="results", S_base=500):
        self.dss_file = dss_file
        self.output_dir = output_dir
        self.S_base = S_base 
        self.load_data = []
        self.bus_index = {}
        self.reference_bus = None
        self.P_scenarios = {}
        self.Q_scenarios = {}
        os.makedirs(self.output_dir, exist_ok=True)
        logging.basicConfig(filename=os.path.join(self.output_dir, 'log.txt'), level=logging.INFO)

    def load_system(self):
        """Load the DSS system file and solve the power flow."""
        try:
            dss.run_command(f'Redirect "{self.dss_file}"')
            dss.Solution.Solve()
            logging.info(f'Successfully loaded and solved DSS file: {self.dss_file}')
        except Exception as e:
            logging.error(f'Error loading DSS file: {e}')
            raise

    def load_line_data(self, line_data_file):
        """Load line data from a JSON file."""
        try:
            with open(line_data_file, "r") as f:
                self.line_data = json.load(f)
            logging.info(f'Successfully loaded line data from {line_data_file}')
        except Exception as e:
            logging.error(f'Error loading line data: {e}')
            raise

    def generate_bus_index_from_lines(self):
        """Generate a bus index from the line data."""
        try:
            buses = []
            seen_buses = set()
            for line, data in self.line_data.items():
                bus1 = data['FromBus']
                bus2 = data['ToBus']
                if bus1 not in seen_buses:
                    buses.append(bus1)
                    seen_buses.add(bus1)
                if bus2 not in seen_buses:
                    buses.append(bus2)
                    seen_buses.add(bus2)

            self.bus_index = {bus: i for i, bus in enumerate(buses)}
            self.reference_bus = buses[0] if buses else None
            logging.info('Successfully generated bus index from line data')
        except Exception as e:
            logging.error(f'Error generating bus index from line data: {e}')
            raise

    def load_scenarios(self, test_case):
        """Load previously saved P and Q scenarios."""
        try:
            self.P_scenarios = np.load(os.path.join(self.output_dir, f"P_OpenDSS_normal_{test_case}.npy"), allow_pickle=True).item()
            self.Q_scenarios = np.load(os.path.join(self.output_dir, f"Q_OpenDSS_normal_{test_case}.npy"), allow_pickle=True).item()
            logging.info(f'Successfully loaded scenarios for {test_case}')
        except Exception as e:
            logging.error(f'Error loading scenarios: {e}')
            raise

    def extract_load_data(self):
        """Extract load data from the DSS system."""
        try:
            loads = dss.Loads.AllNames()
            for load in loads:
                dss.Loads.Name(load)
                bus = dss.CktElement.BusNames()[0].split('.')[0]
                phases = dss.Loads.Phases()
                conn = 'Delta' if dss.Loads.IsDelta() else 'Wye'
                kV = dss.Loads.kV()

                P_scenarios = self.P_scenarios.get(load, [])
                Q_scenarios = self.Q_scenarios.get(load, [])

                for scenario_idx in range(len(P_scenarios)):
                    P = P_scenarios[scenario_idx]
                    Q = Q_scenarios[scenario_idx]
                    S = np.sqrt(P**2 + Q**2)
                    power_factor = P / S if S != 0 else 1.0

                    if bus in self.bus_index:
                        self.load_data.append({
                            'name': load,
                            'bus': bus,
                            'phases': phases,
                            'connection': conn,
                            'P': P,
                            'Q': Q,
                            'kV': kV,
                            'power_factor': power_factor,
                            'bus_full': dss.CktElement.BusNames()[0],
                            'scenario': scenario_idx
                        })
            logging.info('Successfully extracted load data')
        except Exception as e:
            logging.error(f'Error extracting load data: {e}')
            raise

    def organize_power_data(self):
        """Organize power data by bus and phase."""
        try:
            scenario_count = len(next(iter(self.P_scenarios.values())))
            P_bus_phase = {bus: np.zeros((scenario_count, 3)) for bus in self.bus_index}
            Q_bus_phase = {bus: np.zeros((scenario_count, 3)) for bus in self.bus_index}

            for scenario_idx in range(scenario_count):
                for load in self.load_data:
                    if load['scenario'] == scenario_idx:
                        bus = load['bus']
                        connection = load['connection']
                        phase_indices = [int(phase) - 1 for phase in load['bus_full'].split('.')[1:] if phase.isdigit()]

                        P = load['P']
                        Q = load['Q']

                        if connection == 'Delta' and len(phase_indices) == 3:
                            for phase in phase_indices:
                                P_bus_phase[bus][scenario_idx][phase] = P / 3
                                Q_bus_phase[bus][scenario_idx][phase] = Q / 3
                        else:
                            if phase_indices:
                                first_phase_index = phase_indices[0]
                                P_bus_phase[bus][scenario_idx][first_phase_index] = P
                                Q_bus_phase[bus][scenario_idx][first_phase_index] = Q

            logging.info('Successfully organized power data')
            return P_bus_phase, Q_bus_phase
        except Exception as e:
            logging.error(f'Error organizing power data: {e}')
            raise

    def convert_to_pu(self, P_bus_phase, Q_bus_phase):
        """Convert power data to per unit values."""
        try:
            P_bus_phase_pu = {bus: P / self.S_base for bus, P in P_bus_phase.items()}
            Q_bus_phase_pu = {bus: Q / self.S_base for bus, Q in Q_bus_phase.items()}
            logging.info('Successfully converted power data to per unit values')
            return P_bus_phase_pu, Q_bus_phase_pu
        except Exception as e:
            logging.error(f'Error converting power data to per unit values: {e}')
            raise

    def remove_reference_bus_columns(self, P_bus_phase, Q_bus_phase):
        """Remove reference bus columns from the power data."""
        try:
            if self.reference_bus in P_bus_phase:
                del P_bus_phase[self.reference_bus]
            if self.reference_bus in Q_bus_phase:
                del Q_bus_phase[self.reference_bus]
            logging.info(f'Successfully removed reference bus {self.reference_bus} from power data')
        except Exception as e:
            logging.error(f'Error removing reference bus from power data: {e}')
            raise

    def save_power_data(self, P_bus_phase, Q_bus_phase, suffix="", test_case=""):
        """Save power data to JSON files."""
        try:
            P_bus_phase = {bus: P.tolist() for bus, P in P_bus_phase.items()}
            Q_bus_phase = {bus: Q.tolist() for bus, Q in Q_bus_phase.items()}

            with open(os.path.join(self.output_dir, f"{test_case}_P_LDF_normal{suffix}.json"), "w") as fp:
                json.dump(P_bus_phase, fp, indent=4)
            with open(os.path.join(self.output_dir, f"{test_case}_Q_LDF_normal{suffix}.json"), "w") as fq:
                json.dump(Q_bus_phase, fq, indent=4)
            logging.info(f'P and Q data saved to {self.output_dir} as {suffix}')
        except Exception as e:
            logging.error(f'Error saving power data: {e}')
            raise

    def save_load_data(self, test_case=""):
        """Save load data to a JSON file."""
        try:
            with open(os.path.join(self.output_dir, f"{test_case}_load_data.json"), "w") as fl:
                json.dump(self.load_data, fl, indent=4)
            logging.info(f'Load data saved to {self.output_dir}')
        except Exception as e:
            logging.error(f'Error saving load data: {e}')
            raise

    def load_power_data(self, suffix="", test_case=""):
        """Load power data from JSON files."""
        try:
            with open(os.path.join(self.output_dir, f"{test_case}_P_LDF_normal{suffix}.json"), "r") as fp:
                P_bus_phase = json.load(fp)
            with open(os.path.join(self.output_dir, f"{test_case}_Q_LDF_normal{suffix}.json"), "r") as fq:
                Q_bus_phase = json.load(fq)
            logging.info(f'Successfully loaded power data with suffix {suffix}')
            return P_bus_phase, Q_bus_phase
        except Exception as e:
            logging.error(f'Error loading power data: {e}')
            raise

    def power_data_to_matrix(self, power_data, scenario_idx):
        """Convert power data to a matrix format."""
        power_matrix = np.zeros((len(self.bus_index) - 1, 3))  # Adjust for removed reference bus
        try:
            for bus, idx in self.bus_index.items():
                if bus != self.reference_bus and bus in power_data:
                    power_matrix[idx - (1 if idx > self.bus_index[self.reference_bus] else 0), :] = power_data[bus][scenario_idx]
            logging.info('Successfully converted power data to matrix')
            return power_matrix
        except Exception as e:
            logging.error(f'Error converting power data to matrix: {e}')
            raise

if __name__ == "__main__":
    try:
        TestCase = '37Bus'
        #TestCase = '13Bus'

        simulator = LoadScenarioGeneration(
            dss_file=f"/Users/babaktaheri/Desktop/OLDF/Multi-phase/data/IEEETestCases/{TestCase}/ieee37.dss",
            #dss_file=f"/Users/babaktaheri/Desktop/OLDF/Multi-phase/data/IEEETestCases/{TestCase}/IEEE13Nodeckt.dss",
            S_base=500
        )

        simulator.load_system()
        simulator.load_line_data(line_data_file=f"results/{TestCase}_line_data.json")  # Update with correct file path
        simulator.generate_bus_index_from_lines()
        simulator.load_scenarios(TestCase)
        simulator.extract_load_data()
        P_bus_phase, Q_bus_phase = simulator.organize_power_data()
        
        simulator.remove_reference_bus_columns(P_bus_phase, Q_bus_phase)

        P_bus_phase_pu, Q_bus_phase_pu = simulator.convert_to_pu(P_bus_phase, Q_bus_phase)
        
        simulator.save_power_data(P_bus_phase, Q_bus_phase, suffix="", test_case=TestCase)
        simulator.save_power_data(P_bus_phase_pu, Q_bus_phase_pu, suffix="_pu", test_case=TestCase)
        simulator.save_load_data(test_case=TestCase)

        P_bus_phase, Q_bus_phase = simulator.load_power_data(suffix="", test_case=TestCase)
        P_bus_phase_pu, Q_bus_phase_pu = simulator.load_power_data(suffix="_pu", test_case=TestCase)

        scenario_idx = 0
        P_matrix = simulator.power_data_to_matrix(P_bus_phase, scenario_idx)
        Q_matrix = simulator.power_data_to_matrix(Q_bus_phase, scenario_idx)
        P_matrix_pu = simulator.power_data_to_matrix(P_bus_phase_pu, scenario_idx)
        Q_matrix_pu = simulator.power_data_to_matrix(Q_bus_phase_pu, scenario_idx)

        print("Bus index:\n", simulator.bus_index)
    except Exception as e:
        logging.error(f'Error in main execution: {e}')
        print(f'Error in main execution: {e}')


# Training

In [ ]:
import pandas as pd
import numpy as np
from opendssdirect import dss
import os
import matplotlib.pyplot as plt
import json
from scipy.optimize import minimize
from joblib import Parallel, delayed
import random
import time

class OptimizedLindist3FlowTraining:
    def __init__(self, path, test_case, optimization_method):
        self.path = path
        self.test_case = test_case
        self.optimization_method = optimization_method
        self.Ref_V = np.array([0.9890**2, 1.0245**2, 1.0146**2])  # IEEE 37 Bus reference voltages
        #self.Ref_V = np.array([1.056**2, 1.0374**2, 1.056**2]) # IEEE 13 Bus
        self.line_data = None
        self.buses = None
        self.phases = None
        self.bus_index = None
        self.incidence_matrix = None
        self.incidence_df = None
        self.A = None
        self.F = None
        self.block_diagonal_matrix_hp = None
        self.block_diagonal_matrix_hq = None
        self.P_data = None
        self.Q_data = None
        self.V_AC = None

    def load_dss_master_file(self):
        """Load the DSS master file."""
        dss_master_file_path = f"{self.path}/data/IEEETestCases/{self.test_case}Bus/ieee37.dss"
        dss.run_command("Clear")
        dss.run_command(f'Redirect "{dss_master_file_path}"')
        if dss.Circuit.Name() == "":
            raise Exception("No active circuit. Please check the DSS master file.")

    def load_line_data_from_json(self):
        """Load line data from a JSON file."""
        file_path = f"{self.path}/results/{self.test_case}Bus_line_data.json"
        with open(file_path, 'r') as file:
            self.line_data = json.load(file)

    def extract_buses_and_phases(self):
        """Extract bus and phase information from line data."""
        buses = []
        phases = {'1', '2', '3'}
        seen_buses = set()
        for line_info in self.line_data.values():
            bus1_base = line_info['FromBus'].split('.')[0]
            bus2_base = line_info['ToBus'].split('.')[0]
            if bus1_base not in seen_buses:
                buses.append(bus1_base)
                seen_buses.add(bus1_base)
            if bus2_base not in seen_buses:
                buses.append(bus2_base)
                seen_buses.add(bus2_base)
        self.buses = buses
        self.phases = phases
        self.bus_index = {bus: i for i, bus in enumerate(buses)}

    def create_incidence_matrix(self):
        """Create the incidence matrix."""
        incidence_matrix = np.zeros((len(self.buses) * len(self.phases), len(self.line_data) * len(self.phases)), dtype=int)
        for line_idx, (line_id, line_info) in enumerate(self.line_data.items()):
            bus1 = line_info['FromBus']
            bus2 = line_info['ToBus']
            bus1_phases = line_info['Bus1Phases']
            bus2_phases = line_info['Bus2Phases']
            for phase in self.phases:
                if phase in bus1_phases:
                    row_idx_bus1 = self.bus_index[bus1] * 3 + int(phase) - 1
                    incidence_matrix[row_idx_bus1, line_idx * 3 + int(phase) - 1] = 1
                if phase in bus2_phases:
                    row_idx_bus2 = self.bus_index[bus2] * 3 + int(phase) - 1
                    incidence_matrix[row_idx_bus2, line_idx * 3 + int(phase) - 1] = -1
        self.incidence_matrix = incidence_matrix.T

    def prepare_incidence_df(self):
        """Prepare the incidence DataFrame."""
        incidence_df = pd.DataFrame(self.incidence_matrix, columns=[f"{bus}.{phase}" for bus in self.buses for phase in self.phases])
        incidence_df.index = [f"{line_name}.{phase}" for line_name in self.line_data.keys() for phase in self.phases]
        reference_bus = self.buses[0]
        reference_columns = [f"{reference_bus}.{phase}" for phase in self.phases]
        incidence_df.drop(columns=reference_columns, inplace=True)
        self.incidence_df = incidence_df

    def compute_pseudo_inverse(self):
        """Compute the pseudo-inverse of the incidence matrix."""
        A = self.incidence_df.to_numpy()
        F = np.linalg.pinv(A)
        self.A = A
        self.F = F

    def load_data_from_files(self):
        """Load required data from files."""
        self.block_diagonal_matrix_hp = np.load(f'{self.path}/results/{self.test_case}Bus_block_diagonal_matrix_hp.npy')
        self.block_diagonal_matrix_hq = np.load(f'{self.path}/results/{self.test_case}Bus_block_diagonal_matrix_hq.npy')
        with open(f'{self.path}/results/{self.test_case}Bus_P_LDF_normal_pu.json', 'r') as f:
            self.P_data = json.load(f)
        with open(f'{self.path}/results/{self.test_case}Bus_Q_LDF_normal_pu.json', 'r') as f:
            self.Q_data = json.load(f)
        with open(f"{self.path}/results/{self.test_case}Bus_normal_voltages.json", "r") as file:
            self.V_AC = json.load(file)

    def LinDist3Flow(self, H_p, H_q, gamma, rho, varrho, ref_bus, n_buses_3ph, n_branches, F, A, P_inj, Q_inj, V_AC, Batch_size, Ref_V, scenario_idx):
        """Compute the 3-phase LinDistFlow and sensitivities."""
        gamma = gamma.reshape(n_buses_3ph, 1)
        rho = rho.reshape(n_buses_3ph, 1)
        varrho = varrho.reshape(n_buses_3ph, 1)

        Ref_V_expanded = np.tile(Ref_V, n_buses_3ph // 3).reshape(-1, 1)
        v = Ref_V_expanded - F @ H_p @ F.T @ (-P_inj + rho) - F @ H_q @ F.T @ (-Q_inj + varrho) + gamma

        P = A.T @ A
        B = np.zeros((len(A), 1))
        for i in range(len(A)):
            if np.any(P[i, :] != 0):
                B[i] = 1
        v = np.multiply(B, v)

        dvdH_p = -2 * (1 / n_buses_3ph) * (1 / Batch_size) * np.kron(F.T,  (F.T @ (-P_inj + rho)))
        dvdH_q = -2 * (1 / n_buses_3ph) * (1 / Batch_size) * np.kron(F.T,  (F.T @ (-Q_inj + varrho)))
        dvdrho = -2 * (1 / n_buses_3ph) * (1 / Batch_size) * F @ H_p @ F.T
        dvdvarrho = -2 * (1 / n_buses_3ph) * (1 / Batch_size) * F @ H_q @ F.T
        dvdgamma = (1 / n_buses_3ph) * (1 / Batch_size) * np.eye(n_buses_3ph)

        v_AC = np.array(V_AC[f"scenario_{scenario_idx}"])
        v_AC = v_AC[3:]
        v_AC = np.array(v_AC).reshape((n_buses_3ph, 1))
        v = np.reshape(v, (n_buses_3ph, 1))

        gradient_H_p = dvdH_p @ (v - v_AC**2)
        gradient_H_q = dvdH_q @ (v - v_AC**2)
        gradient_rho = dvdrho @ (v - v_AC**2)
        gradient_varrho = dvdvarrho @ (v - v_AC**2)
        gradient_gamma = dvdgamma @ (v - v_AC**2)

        return v, v_AC, gradient_H_p, gradient_H_q, gradient_gamma, gradient_rho, gradient_varrho

    def objective_function(self, params):
        """Objective function for the optimization process."""
        global gradient_Dr, gradient_Dx, gradient_gamma_P, gradient_gamma_Q, gradient_bias, v, v_AC

        n_buses_3ph = self.A.shape[0]
        n_branches = n_buses_3ph // 3

        H_p = params[:len(self.block_diagonal_matrix_hp)**2].reshape((len(self.block_diagonal_matrix_hp), len(self.block_diagonal_matrix_hp)))
        H_q = params[len(self.block_diagonal_matrix_hp)**2:2 * len(self.block_diagonal_matrix_hp)**2].reshape((len(self.block_diagonal_matrix_hp), len(self.block_diagonal_matrix_hp)))
        gamma = params[2 * len(self.block_diagonal_matrix_hp)**2:2 * len(self.block_diagonal_matrix_hp)**2 + n_buses_3ph].reshape((n_buses_3ph, 1))
        rho = params[2 * len(self.block_diagonal_matrix_hp)**2 + n_buses_3ph:2 * len(self.block_diagonal_matrix_hp)**2 + 2 * n_buses_3ph].reshape((n_buses_3ph, 1))
        varrho = params[2 * len(self.block_diagonal_matrix_hp)**2 + 2 * n_buses_3ph:].reshape((n_buses_3ph, 1))

        ref_bus = 0
        Batch_size = 20
        scen_start = 1
        scen_final = 21
        v_results, v_AC_results, gradient_H_p, gradient_H_q, gradient_gamma, gradient_rho, gradient_varrho = zip(*Parallel(n_jobs=-1)(delayed(self.LinDist3Flow)(
            H_p, H_q, gamma, rho, varrho, ref_bus, n_buses_3ph, n_branches, self.F, self.A, np.array([values[scenario_idx] for values in self.P_data.values()]).flatten().reshape(-1, 1), np.array([values[scenario_idx] for values in self.Q_data.values()]).flatten().reshape(-1, 1), self.V_AC, Batch_size, self.Ref_V, s) for s in random.sample(range(scen_start-1, scen_final-1), Batch_size)))

        Gradient_Dr = np.sum(gradient_H_p, axis=0)
        Gradient_Dx = np.sum(gradient_H_q, axis=0)
        Gradient_gamma_P = np.sum(gradient_rho, axis=0)
        Gradient_gamma_Q = np.sum(gradient_varrho, axis=0)
        Gradient_bias = np.sum(gradient_gamma, axis=0)

        Jac = np.concatenate((Gradient_Dr, Gradient_Dx, Gradient_gamma_P, Gradient_gamma_Q, Gradient_bias))
        V_LDF = np.concatenate(v_results)
        V_DF = np.concatenate(v_AC_results)

        objective = (1 / n_buses_3ph) * (1 / Batch_size) * np.sum((V_LDF - V_DF**2)**2)

        return objective, Jac.flatten()

    def optimize(self):
        """Optimize the parameters."""
        H_p = self.block_diagonal_matrix_hp.flatten()
        H_q = self.block_diagonal_matrix_hq.flatten()
        rho = np.zeros(self.A.shape[0])
        varrho = np.zeros(self.A.shape[0])
        gamma = np.zeros(self.A.shape[0])

        initial_params = np.concatenate((H_p, H_q, rho, varrho, gamma))

        result = minimize(
            fun=self.objective_function,
            x0=initial_params,
            jac=True,
            method=self.optimization_method,
            options={'gtol': 1e-16, 'disp': True}
        )
        return result

    def save_results(self, result):
        """Save the optimization results."""
        optimized_params = result.x

        H_p = optimized_params[:len(self.block_diagonal_matrix_hp)**2]
        H_q = optimized_params[len(self.block_diagonal_matrix_hp)**2:2 * len(self.block_diagonal_matrix_hp)**2]
        gamma_P_MIN = optimized_params[2 * len(self.block_diagonal_matrix_hp)**2:2 * len(self.block_diagonal_matrix_hp)**2 + self.A.shape[0]]
        gamma_Q_MIN = optimized_params[2 * len(self.block_diagonal_matrix_hp)**2 + self.A.shape[0]:2 * len(self.block_diagonal_matrix_hp)**2 + 2 * self.A.shape[0]]
        bias_Min = optimized_params[2 * len(self.block_diagonal_matrix_hp)**2 + 2 * self.A.shape[0]:]

        np.savetxt(f"{self.path}/parameters/H_p_{self.test_case}bus_{self.optimization_method}.txt", H_p, fmt='%f')
        np.savetxt(f"{self.path}/parameters/H_q_{self.test_case}bus_{self.optimization_method}.txt", H_q, fmt='%f')
        np.savetxt(f"{self.path}/parameters/gamma_P_{self.test_case}bus_{self.optimization_method}.txt", gamma_P_MIN, fmt='%f')
        np.savetxt(f"{self.path}/parameters/gamma_Q_{self.test_case}bus_{self.optimization_method}.txt", gamma_Q_MIN, fmt='%f')
        np.savetxt(f"{self.path}/parameters/bias_{self.test_case}bus_{self.optimization_method}.txt", bias_Min, fmt='%f')

def main():
    path = '/Users/babaktaheri/Desktop/OLDF/Multi-phase'
    test_case = '37'
    optimization_method = 'TNC'
    voltage_optimizer = OptimizedLindist3FlowTraining(path, test_case, optimization_method)
    
    voltage_optimizer.load_dss_master_file()
    voltage_optimizer.load_line_data_from_json()
    voltage_optimizer.extract_buses_and_phases()
    voltage_optimizer.create_incidence_matrix()
    voltage_optimizer.prepare_incidence_df()
    voltage_optimizer.compute_pseudo_inverse()
    voltage_optimizer.load_data_from_files()
    
    start_time = time.process_time()
    result = voltage_optimizer.optimize()
    end_time = time.process_time()
    
    execution_time = end_time - start_time
    print(f'Total execution time: {execution_time} seconds')
    
    voltage_optimizer.save_results(result)

if __name__ == "__main__":
    main()


# Train by leveragating the block diagonal structure of H matrices

In [ ]:
import pandas as pd
import numpy as np
from opendssdirect import dss
import os
import matplotlib.pyplot as plt
import json
from scipy.optimize import minimize
import random
import time

class OLDFWithBlockDiagonal:
    def __init__(self, test_case, optimization_method, path):
        self.TEST_CASE = test_case
        self.OPTIMIZATION_METHOD = optimization_method
        self.PATH = path
        self.Ref_V = None
        self.line_data = None
        self.buses = None
        self.phases = None
        self.bus_index = None
        self.incidence_df = None
        self.A = None
        self.F = None
        self.block_diagonal_matrix_hp = None
        self.block_diagonal_matrix_hq = None
        self.P_data = None
        self.Q_data = None
        self.V_AC = None

    def load_dss_master_file(self, dss_master_file_path):
        dss.run_command("Clear")
        dss.run_command(f'Redirect "{dss_master_file_path}"')
        if dss.Circuit.Name() == "":
            raise Exception("No active circuit. Please check the DSS master file.")

    def load_line_data_from_json(self, file_path):
        with open(file_path, 'r') as file:
            self.line_data = json.load(file)

    def extract_buses_and_phases(self):
        self.buses = []
        self.phases = {'1', '2', '3'}
        seen_buses = set()
        for line_info in self.line_data.values():
            bus1_base = line_info['FromBus'].split('.')[0]
            bus2_base = line_info['ToBus'].split('.')[0]
            if bus1_base not in seen_buses:
                self.buses.append(bus1_base)
                seen_buses.add(bus1_base)
            if bus2_base not in seen_buses:
                self.buses.append(bus2_base)
                seen_buses.add(bus2_base)
        self.bus_index = {bus: i for i, bus in enumerate(self.buses)}

    def create_incidence_matrix(self):
        incidence_matrix = np.zeros((len(self.buses) * len(self.phases), len(self.line_data) * len(self.phases)), dtype=int)
        for line_idx, (line_id, line_info) in enumerate(self.line_data.items()):
            bus1 = line_info['FromBus']
            bus2 = line_info['ToBus']
            bus1_phases = line_info['Bus1Phases']
            bus2_phases = line_info['Bus2Phases']
            for phase in self.phases:
                if phase in bus1_phases:
                    row_idx_bus1 = self.bus_index[bus1] * 3 + int(phase) - 1
                    incidence_matrix[row_idx_bus1, line_idx * 3 + int(phase) - 1] = 1
                if phase in bus2_phases:
                    row_idx_bus2 = self.bus_index[bus2] * 3 + int(phase) - 1
                    incidence_matrix[row_idx_bus2, line_idx * 3 + int(phase) - 1] = -1
        return incidence_matrix.T

    def prepare_incidence_df(self, incidence_matrix):
        incidence_df = pd.DataFrame(incidence_matrix, columns=[f"{bus}.{phase}" for bus in self.buses for phase in self.phases])
        incidence_df.index = [f"{line_name}.{phase}" for line_name in self.line_data.keys() for phase in self.phases]
        reference_bus = self.buses[0]
        reference_columns = [f"{reference_bus}.{phase}" for phase in self.phases]
        incidence_df.drop(columns=reference_columns, inplace=True)
        self.incidence_df = incidence_df

    def compute_pseudo_inverse(self):
        A = self.incidence_df.to_numpy()
        F = np.linalg.pinv(A)
        self.A = A
        self.F = F

    def load_data_from_files(self):
        self.block_diagonal_matrix_hp = np.load(f'{self.PATH}/results/{self.TEST_CASE}Bus_block_diagonal_matrix_hp.npy')
        self.block_diagonal_matrix_hq = np.load(f'{self.PATH}/results/{self.TEST_CASE}Bus_block_diagonal_matrix_hq.npy')
        with open(f'{self.PATH}/results/{self.TEST_CASE}Bus_P_LDF_normal_pu.json', 'r') as f:
            self.P_data = json.load(f)
        with open(f'{self.PATH}/results/{self.TEST_CASE}Bus_Q_LDF_normal_pu.json', 'r') as f:
            self.Q_data = json.load(f)

    def extract_block_diagonal_elements(self, block_diagonal_matrix):
        block_size = 3
        num_blocks = block_diagonal_matrix.shape[0] // block_size
        blocks = []
        
        for i in range(num_blocks):
            start_index = i * block_size
            end_index = start_index + block_size
            
            block = block_diagonal_matrix[start_index:end_index, start_index:end_index]
            blocks.append(block)
        
        concatenated_blocks = np.concatenate([block.flatten() for block in blocks])
        return concatenated_blocks

    def reconstruct_block_diagonal_matrix(self, concatenated_blocks, block_size, num_blocks):
        expected_length = block_size * block_size * num_blocks
        if len(concatenated_blocks) != expected_length:
            raise ValueError(f"Expected length {expected_length}, but got {len(concatenated_blocks)}")
        
        reshaped_blocks = np.reshape(concatenated_blocks, (num_blocks, block_size, block_size))
        initial_matrix_size = block_size * num_blocks
        initial_matrix = np.zeros((initial_matrix_size, initial_matrix_size))
        
        for i in range(num_blocks):
            start_index = i * block_size
            end_index = start_index + block_size
            
            block = reshaped_blocks[i]
            initial_matrix[start_index:end_index, start_index:end_index] = block
        
        return initial_matrix

    def LinDist3Flow(self, H_p_blocks, H_q_blocks, gamma, rho, varrho, ref_bus, n_buses_3ph, n_branches, F, A, P_inj, Q_inj, V_AC, Batch_size, Ref_V, scenario_idx):
        block_size  = 3
        num_blocks  = n_branches

        H_p = self.reconstruct_block_diagonal_matrix(H_p_blocks, block_size, num_blocks)
        H_q = self.reconstruct_block_diagonal_matrix(H_q_blocks, block_size, num_blocks)

        gamma = gamma.reshape(n_buses_3ph, 1)
        rho = rho.reshape(n_buses_3ph, 1)
        varrho = varrho.reshape(n_buses_3ph, 1)

        Ref_V_expanded = np.tile(Ref_V, n_buses_3ph // 3).reshape(-1, 1)
        v = Ref_V_expanded - F @ H_p @ F.T @ (-P_inj + rho) - F @ H_q @ F.T @ (-Q_inj + varrho) + gamma

        P = A.T @ A
        B = np.zeros((len(A), 1))
        for i in range(len(A)):
            if np.any(P[i, :] != 0):
                B[i] = 1
        v = np.multiply(B, v)

        dvdH_p = -2 * (1 / n_buses_3ph) * (1 / Batch_size) * np.kron(F.T, (F.T @ (-P_inj + rho)))
        dvdH_q = -2 * (1 / n_buses_3ph) * (1 / Batch_size) * np.kron(F.T, (F.T @ (-Q_inj + varrho)))
        dvdrho = -2 * (1 / n_buses_3ph) * (1 / Batch_size) * F @ H_p @ F.T
        dvdvarrho = -2 * (1 / n_buses_3ph) * (1 / Batch_size) * F @ H_q @ F.T
        dvdgamma = (1 / n_buses_3ph) * (1 / Batch_size) * np.eye(n_buses_3ph)

        v_AC = np.array(V_AC[f"scenario_{scenario_idx}"])
        v_AC = v_AC[3:]  # delete the reference bus
        v_AC = np.array(v_AC).reshape((n_buses_3ph, 1))
        v = np.reshape(v, (n_buses_3ph, 1))

        gradient_H_p = dvdH_p @ (v - v_AC**2)
        gradient_H_q = dvdH_q @ (v - v_AC**2)
        gradient_rho = dvdrho @ (v - v_AC**2)
        gradient_varrho = dvdvarrho @ (v - v_AC**2)
        gradient_gamma = dvdgamma @ (v - v_AC**2)

        return v, v_AC, gradient_H_p, gradient_H_q, gradient_gamma, gradient_rho, gradient_varrho

    def objective_function(self, params, A, F, P_data, Q_data, Ref_V, V_AC, Batch_size, scen_start, scen_final):
        Gradient_Dr = 0
        Gradient_Dx = 0
        Gradient_gamma_P = 0
        Gradient_gamma_Q = 0
        Gradient_bias = 0

        V_LDF = []
        V_DF = []
        n_buses_3ph = A.shape[0]
        n_branches = n_buses_3ph // 3

        size_blocks = n_branches * 9
        H_p_blocks = params[:size_blocks].reshape((size_blocks, 1))  
        H_q_blocks = params[size_blocks:2*size_blocks].reshape((size_blocks, 1))

        gamma  = params[2*size_blocks:2*size_blocks + n_buses_3ph].reshape((n_buses_3ph, 1))
        rho    = params[2*size_blocks + n_buses_3ph:2*size_blocks + 2 * n_buses_3ph].reshape((n_buses_3ph, 1))
        varrho = params[2*size_blocks + 2 * n_buses_3ph:].reshape((n_buses_3ph, 1))

        scenario_indices = random.sample(range(scen_start - 1, scen_final - 1), Batch_size)
        ref_bus = 0
        for idx in scenario_indices:
            v_result, v_AC_result, gradient_H_p, gradient_H_q, gradient_gamma, gradient_rho, gradient_varrho = self.LinDist3Flow(
                H_p_blocks, H_q_blocks, gamma, rho, varrho, ref_bus, n_buses_3ph, n_branches, F, A, 
                np.array([values[idx] for values in P_data.values()]).flatten().reshape(-1, 1), 
                np.array([values[idx] for values in Q_data.values()]).flatten().reshape(-1, 1), V_AC, Batch_size, Ref_V, idx)
            V_LDF.append(v_result)
            V_DF.append(v_AC_result)
            Gradient_Dr += gradient_H_p
            Gradient_Dx += gradient_H_q
            Gradient_gamma_P += gradient_rho
            Gradient_gamma_Q += gradient_varrho
            Gradient_bias += gradient_gamma

        V_LDF = np.concatenate(V_LDF)
        V_DF = np.concatenate(V_DF)

        Gradient_Dr = Gradient_Dr.reshape((A.shape[0], A.shape[0]))
        Gradient_Dx = Gradient_Dx.reshape((A.shape[0], A.shape[0]))

        Gradient_Dr = self.extract_block_diagonal_elements(Gradient_Dr)
        Gradient_Dx = self.extract_block_diagonal_elements(Gradient_Dx)  

        Gradient_Dr = Gradient_Dr.reshape(-1, 1)
        Gradient_Dx = Gradient_Dx.reshape(-1, 1)

        Gradient_gamma_P = Gradient_gamma_P.reshape(-1, 1)
        Gradient_gamma_Q = Gradient_gamma_Q.reshape(-1, 1)
        Gradient_bias = Gradient_bias.reshape(-1, 1)

        Jac = np.concatenate((Gradient_Dr, Gradient_Dx, Gradient_gamma_P, Gradient_gamma_Q, Gradient_bias))

        objective = (1 / n_buses_3ph) * (1 / Batch_size) * np.sum((V_LDF - V_DF**2)**2)

        return objective, Jac.flatten()


    def run_optimization(self):
        dss_master_file_path = f"{self.PATH}/data/IEEETestCases/{self.TEST_CASE}Bus/ieee37.dss"
        self.load_dss_master_file(dss_master_file_path)

        self.Ref_V = np.array([0.9890**2, 1.0245**2, 1.0146**2])  # IEEE 37 Bus

        self.load_line_data_from_json(f"{self.PATH}/results/{self.TEST_CASE}Bus_line_data.json")
        self.extract_buses_and_phases()
        incidence_matrix = self.create_incidence_matrix()
        self.prepare_incidence_df(incidence_matrix)
        self.compute_pseudo_inverse()
        self.load_data_from_files()

        with open(f"results/{self.TEST_CASE}Bus_normal_voltages.json", "r") as file:
            self.V_AC = json.load(file)
            
        ref_bus    = 0
        scen_start = 1
        scen_final = 21
        Batch_size = 20

        H_p = self.extract_block_diagonal_elements(self.block_diagonal_matrix_hp)
        H_q = self.extract_block_diagonal_elements(self.block_diagonal_matrix_hq)
        rho = np.zeros(self.A.shape[0])
        varrho = np.zeros(self.A.shape[0])
        gamma = np.zeros(self.A.shape[0])

        initial_params = np.concatenate((H_p, H_q, rho, varrho, gamma))

        start_time = time.process_time()

        result = minimize(
            fun=self.objective_function,
            x0=initial_params,
            args=(self.A, self.F, self.P_data, self.Q_data, self.Ref_V, self.V_AC, Batch_size, scen_start, scen_final),
            jac=True,
            method=self.OPTIMIZATION_METHOD,
            options={'gtol': 1e-16, 'disp': True}
        )

        optimized_params = result.x
        end_time = time.process_time()
        execution_time = end_time - start_time
        print(f'Total execution time: {execution_time} seconds')

        size_hp = 3 * self.A.shape[0]
        H_p = optimized_params[:size_hp]
        H_q = optimized_params[size_hp:2*size_hp]
        gamma_P_MIN = optimized_params[2*size_hp:2*size_hp + self.A.shape[0]]
        gamma_Q_MIN = optimized_params[2*size_hp + self.A.shape[0]:2*size_hp + 2*self.A.shape[0]]
        bias_Min = optimized_params[2*size_hp + 2*self.A.shape[0]:]

        print(f'Best H_p {self.TEST_CASE}bus:\n', H_p)
        print(f'Best H_q {self.TEST_CASE}bus:\n', H_q)
        print(f'Best gamma_P {self.TEST_CASE}bus:\n', gamma_P_MIN)
        print(f'Best gamma_Q {self.TEST_CASE}bus:\n', gamma_Q_MIN)
        print(f'Best bias {self.TEST_CASE}bus\n:', bias_Min)

        np.savetxt(f"{self.PATH}/parameters/H_p_{self.TEST_CASE}bus_{self.OPTIMIZATION_METHOD}.txt", H_p, fmt='%f')
        np.savetxt(f"{self.PATH}/parameters/H_q_{self.TEST_CASE}bus_{self.OPTIMIZATION_METHOD}.txt", H_q, fmt='%f')
        np.savetxt(f"{self.PATH}/parameters/gamma_P_{self.TEST_CASE}bus_{self.OPTIMIZATION_METHOD}.txt", gamma_P_MIN, fmt='%f')
        np.savetxt(f"{self.PATH}/parameters/gamma_Q_{self.TEST_CASE}bus_{self.OPTIMIZATION_METHOD}.txt", gamma_Q_MIN, fmt='%f')
        np.savetxt(f"{self.PATH}/parameters/bias_{self.TEST_CASE}bus_{self.OPTIMIZATION_METHOD}.txt", bias_Min, fmt='%f')

if __name__ == "__main__":
    test_case = '37'  # Replace with '13' if needed
    optimization_method = 'TNC'
    path = '/Users/babaktaheri/Desktop/OLDF/Multi-phase'
    
    optimizer = OLDFWithBlockDiagonal(test_case, optimization_method, path)
    optimizer.run_optimization()


# Read uniform scenarios and make them ready for testing LinDist3Flow

In [ ]:
import numpy as np
from opendssdirect import dss
import os
import json
import logging

class LoadScenarioGeneration:
    def __init__(self, dss_file, output_dir="results", S_base=500):
        self.dss_file = dss_file
        self.output_dir = output_dir
        self.S_base = S_base
        self.load_data = []
        self.bus_index = {}
        self.reference_bus = None
        self.P_scenarios = {}
        self.Q_scenarios = {}
        os.makedirs(self.output_dir, exist_ok=True)
        logging.basicConfig(filename=os.path.join(self.output_dir, 'log.txt'), level=logging.INFO)

    def load_system(self):
        try:
            dss.run_command(f'Redirect "{self.dss_file}"')
            dss.Solution.Solve()
            logging.info(f'Successfully loaded and solved DSS file: {self.dss_file}')
        except Exception as e:
            logging.error(f'Error loading DSS file: {e}')
            raise

    def load_line_data(self, line_data_file):
        try:
            with open(line_data_file, "r") as f:
                self.line_data = json.load(f)
            logging.info(f'Successfully loaded line data from {line_data_file}')
        except Exception as e:
            logging.error(f'Error loading line data: {e}')
            raise

    def generate_bus_index_from_lines(self):
        try:
            buses = []  # Use a list to maintain the order
            seen_buses = set()  # Helper set to avoid duplicates while maintaining order
            for line, data in self.line_data.items():
                bus1 = data['FromBus']
                bus2 = data['ToBus']
                if bus1 not in seen_buses:
                    buses.append(bus1)
                    seen_buses.add(bus1)
                if bus2 not in seen_buses:
                    buses.append(bus2)
                    seen_buses.add(bus2)

            self.bus_index = {bus: i for i, bus in enumerate(buses)}

            # Identify reference bus (typically the first one unless specified otherwise)
            self.reference_bus = buses[0] if buses else None
            logging.info('Successfully generated bus index from line data')
        except Exception as e:
            logging.error(f'Error generating bus index from line data: {e}')
            raise

    def load_scenarios(self, test_case):
        try:
            self.P_scenarios = np.load(os.path.join(self.output_dir, f"P_OpenDSS_uniform_{test_case}.npy"), allow_pickle=True).item()
            self.Q_scenarios = np.load(os.path.join(self.output_dir, f"Q_OpenDSS_uniform_{test_case}.npy"), allow_pickle=True).item()

            logging.info(f'Successfully loaded scenarios for {test_case}')
        except Exception as e:
            logging.error(f'Error loading scenarios: {e}')
            raise

    def extract_load_data(self):
        try:
            loads = dss.Loads.AllNames()
            for load in loads:
                dss.Loads.Name(load)
                bus = dss.CktElement.BusNames()[0].split('.')[0]  # Get only the base bus name
                phases = dss.Loads.Phases()
                conn = 'Delta' if dss.Loads.IsDelta() else 'Wye'
                kV = dss.Loads.kV()

                P_scenarios = self.P_scenarios.get(load, [])
                Q_scenarios = self.Q_scenarios.get(load, [])

                for scenario_idx in range(len(P_scenarios)):
                    P = P_scenarios[scenario_idx]
                    Q = Q_scenarios[scenario_idx]
                    #print(f"Load: {load}, Scenario: {scenario_idx}, P: {P}, Q: {Q}")

                    S = np.sqrt(P**2 + Q**2)  # Apparent power
                    power_factor = P / S if S != 0 else 1.0  # Power factor

                    if bus in self.bus_index:  # Ensure the bus is part of the indexed buses
                        self.load_data.append({
                            'name': load,
                            'bus': bus,
                            'phases': phases,
                            'connection': conn,
                            'P': P,
                            'Q': Q,
                            'kV': kV,
                            'power_factor': power_factor,
                            'bus_full': dss.CktElement.BusNames()[0],  # Full bus name with phases
                            'scenario': scenario_idx
                        })
            logging.info('Successfully extracted load data')
        except Exception as e:
            logging.error(f'Error extracting load data: {e}')
            raise

    def organize_power_data(self):
        try:
            scenario_count = len(next(iter(self.P_scenarios.values())))  # Assuming uniform scenario counts
            P_bus_phase = {bus: np.zeros((scenario_count, 3)) for bus in self.bus_index}
            Q_bus_phase = {bus: np.zeros((scenario_count, 3)) for bus in self.bus_index}

            for scenario_idx in range(scenario_count):
                for load in self.load_data:
                    if load['scenario'] == scenario_idx:
                        bus = load['bus']
                        connection = load['connection']
                        phase_indices = [int(phase) - 1 for phase in load['bus_full'].split('.')[1:] if phase.isdigit()]

                        P = load['P']
                        Q = load['Q']

                        if connection == 'Delta' and len(phase_indices) == 3:
                            # If three-phase Delta, distribute the load evenly across all three phases
                            for phase in phase_indices:
                                P_bus_phase[bus][scenario_idx][phase] = P / 3
                                Q_bus_phase[bus][scenario_idx][phase] = Q / 3
                        else:
                            # Otherwise, assign all the load to the first appearing phase
                            if phase_indices:
                                first_phase_index = phase_indices[0]
                                P_bus_phase[bus][scenario_idx][first_phase_index] = P
                                Q_bus_phase[bus][scenario_idx][first_phase_index] = Q

            logging.info('Successfully organized power data')
            return P_bus_phase, Q_bus_phase
        except Exception as e:
            logging.error(f'Error organizing power data: {e}')
            raise

    def convert_to_pu(self, P_bus_phase, Q_bus_phase):
        try:
            P_bus_phase_pu = {bus: P / self.S_base for bus, P in P_bus_phase.items()}
            Q_bus_phase_pu = {bus: Q / self.S_base for bus, Q in Q_bus_phase.items()}
            logging.info('Successfully converted power data to per unit values')
            return P_bus_phase_pu, Q_bus_phase_pu
        except Exception as e:
            logging.error(f'Error converting power data to per unit values: {e}')
            raise

    def remove_reference_bus_columns(self, P_bus_phase, Q_bus_phase):
        try:
            if self.reference_bus in P_bus_phase:
                del P_bus_phase[self.reference_bus]
            if self.reference_bus in Q_bus_phase:
                del Q_bus_phase[self.reference_bus]
            logging.info(f'Successfully removed reference bus {self.reference_bus} from power data')
        except Exception as e:
            logging.error(f'Error removing reference bus from power data: {e}')
            raise

    def save_power_data(self, P_bus_phase, Q_bus_phase, suffix="", test_case=""):
        try:
            P_bus_phase = {bus: P.tolist() for bus, P in P_bus_phase.items()}
            Q_bus_phase = {bus: Q.tolist() for bus, Q in Q_bus_phase.items()}

            with open(os.path.join(self.output_dir, f"{test_case}_P_LDF_uniform{suffix}.json"), "w") as fp:
                json.dump(P_bus_phase, fp, indent=4)
            with open(os.path.join(self.output_dir, f"{test_case}_Q_LDF_uniform{suffix}.json"), "w") as fq:
                json.dump(Q_bus_phase, fq, indent=4)
            logging.info(f'P and Q data saved to {self.output_dir} as {suffix}')
        except Exception as e:
            logging.error(f'Error saving power data: {e}')
            raise

    def save_load_data(self, test_case=""):
        try:
            with open(os.path.join(self.output_dir, f"{test_case}_load_data.json"), "w") as fl:
                json.dump(self.load_data, fl, indent=4)
            logging.info(f'Load data saved to {self.output_dir}')
        except Exception as e:
            logging.error(f'Error saving load data: {e}')
            raise

    def load_power_data(self, suffix="", test_case=""):
        try:
            with open(os.path.join(self.output_dir, f"{test_case}_P_LDF_uniform{suffix}.json"), "r") as fp:
                P_bus_phase = json.load(fp)
            with open(os.path.join(self.output_dir, f"{test_case}_Q_LDF_uniform{suffix}.json"), "r") as fq:
                Q_bus_phase = json.load(fq)
            logging.info(f'Successfully loaded power data with suffix {suffix}')
            return P_bus_phase, Q_bus_phase
        except Exception as e:
            logging.error(f'Error loading power data: {e}')
            raise

    def power_data_to_matrix(self, power_data, scenario_idx):
        power_matrix = np.zeros((len(self.bus_index) - 1, 3))  # Adjust for removed reference bus
        try:
            for bus, idx in self.bus_index.items():
                if bus != self.reference_bus and bus in power_data:
                    power_matrix[idx - (1 if idx > self.bus_index[self.reference_bus] else 0), :] = power_data[bus][scenario_idx]
            logging.info('Successfully converted power data to matrix')
            return power_matrix
        except Exception as e:
            logging.error(f'Error converting power data to matrix: {e}')
            raise

if __name__ == "__main__":
    try:
        #TestCase = '13Bus'
        TestCase = '37Bus'

        #simulator = LoadScenarioGeneration(dss_file=f"/Users/babaktaheri/Desktop/OLDF/Multi-phase/data/IEEETestCases/{TestCase}/IEEE13Nodeckt.dss", S_base=500.0)
        simulator = LoadScenarioGeneration(dss_file=f"/Users/babaktaheri/Desktop/OLDF/Multi-phase/data/IEEETestCases/{TestCase}/ieee37.dss", S_base=500.0)

        simulator.load_system()
        simulator.load_line_data(line_data_file=f"results/{TestCase}_line_data.json")  # Update with correct file path
        simulator.generate_bus_index_from_lines()
        simulator.load_scenarios(TestCase)
        simulator.extract_load_data()
        P_bus_phase, Q_bus_phase = simulator.organize_power_data()
        
        # Remove reference bus columns
        simulator.remove_reference_bus_columns(P_bus_phase, Q_bus_phase)

        # Convert to per unit values
        P_bus_phase_pu, Q_bus_phase_pu = simulator.convert_to_pu(P_bus_phase, Q_bus_phase)
        
        simulator.save_power_data(P_bus_phase, Q_bus_phase, suffix="", test_case=TestCase)
        simulator.save_power_data(P_bus_phase_pu, Q_bus_phase_pu, suffix="_pu", test_case=TestCase)
        simulator.save_load_data(test_case=TestCase)

        P_bus_phase, Q_bus_phase = simulator.load_power_data(suffix="", test_case=TestCase)
        P_bus_phase_pu, Q_bus_phase_pu = simulator.load_power_data(suffix="_pu", test_case=TestCase)

        # Example: Displaying the power data matrix for the first scenario (index 0)
        scenario_idx = 0
        P_matrix = simulator.power_data_to_matrix(P_bus_phase, scenario_idx)
        Q_matrix = simulator.power_data_to_matrix(Q_bus_phase, scenario_idx)
        P_matrix_pu = simulator.power_data_to_matrix(P_bus_phase_pu, scenario_idx)
        Q_matrix_pu = simulator.power_data_to_matrix(Q_bus_phase_pu, scenario_idx)

        #print(f"P matrix for scenario {scenario_idx} (in kW):\n", P_matrix)
        #print(f"Q matrix for scenario {scenario_idx} (in kVar):\n", Q_matrix)
        #print(f"P matrix for scenario {scenario_idx} (in pu):\n", P_matrix_pu)
        #print(f"Q matrix for scenario {scenario_idx} (in pu):\n", Q_matrix_pu)
        print("Bus index:\n", simulator.bus_index)
    except Exception as e:
        logging.error(f'Error in main execution: {e}')
        print(f'Error in main execution: {e}')


# Test

In [ ]:
import pandas as pd
import numpy as np
from opendssdirect import dss
import time
import os
import matplotlib.pyplot as plt
import json
from scipy.optimize import minimize
from joblib import Parallel, delayed
import random

class OLDFTesting:
    def __init__(self, test_case='37', optimization_method='TNC', path='/Users/babaktaheri/Desktop/OLDF/Multi-phase'):
        self.test_case = test_case
        self.optimization_method = optimization_method
        self.path = path
        self.dss_master_file_path = f"{self.path}/data/IEEETestCases/{self.test_case}Bus/ieee37.dss"
        #self.reference_voltages = np.array([1.056, 1.0374, 1.056]) #IEEE 13Bus
        self.reference_voltages = np.array([0.9890, 1.0245, 1.0146])  # IEEE 37 Bus
        self.load_system()
        self.line_data = self.load_line_data_from_json(f"{self.path}/results/{self.test_case}Bus_line_data.json")
        self.buses, self.phases, self.bus_index = self.extract_buses_and_phases(self.line_data)
        self.incidence_matrix = self.create_incidence_matrix(self.buses, self.phases, self.bus_index, self.line_data)
        self.incidence_df = self.prepare_incidence_df(self.incidence_matrix, self.buses, self.phases, self.line_data)
        self.A, self.F = self.compute_pseudo_inverse(self.incidence_df)
        self.block_diagonal_matrix_hp, self.block_diagonal_matrix_hq, self.P_data, self.Q_data = self.load_data_from_files()
        self.scen_start = 1
        self.scen_final = 10000
        self.H_p = self.block_diagonal_matrix_hp
        self.H_q = self.block_diagonal_matrix_hq
        self.rho = np.zeros(self.A.shape[0])
        self.varrho = np.zeros(self.A.shape[0])
        self.gamma = np.zeros(self.A.shape[0])

    def load_system(self):
        dss.run_command("Clear")
        dss.run_command(f'Redirect "{self.dss_master_file_path}"')
        if dss.Circuit.Name() == "":
            raise Exception("No active circuit. Please check the DSS master file.")

    @staticmethod
    def load_line_data_from_json(file_path):
        with open(file_path, 'r') as file:
            line_data = json.load(file)
        return line_data

    @staticmethod
    def extract_buses_and_phases(line_data):
        buses = []
        phases = {'1', '2', '3'}
        seen_buses = set()
        for line_info in line_data.values():
            bus1_base = line_info['FromBus'].split('.')[0]
            bus2_base = line_info['ToBus'].split('.')[0]
            if bus1_base not in seen_buses:
                buses.append(bus1_base)
                seen_buses.add(bus1_base)
            if bus2_base not in seen_buses:
                buses.append(bus2_base)
                seen_buses.add(bus2_base)
        bus_index = {bus: i for i, bus in enumerate(buses)}
        return buses, phases, bus_index

    @staticmethod
    def create_incidence_matrix(buses, phases, bus_index, line_data):
        incidence_matrix = np.zeros((len(buses) * len(phases), len(line_data) * len(phases)), dtype=int)
        for line_idx, (line_id, line_info) in enumerate(line_data.items()):
            bus1 = line_info['FromBus']
            bus2 = line_info['ToBus']
            bus1_phases = line_info['Bus1Phases']
            bus2_phases = line_info['Bus2Phases']
            for phase in phases:
                if phase in bus1_phases:
                    row_idx_bus1 = bus_index[bus1] * 3 + int(phase) - 1
                    incidence_matrix[row_idx_bus1, line_idx * 3 + int(phase) - 1] = 1
                if phase in bus2_phases:
                    row_idx_bus2 = bus_index[bus2] * 3 + int(phase) - 1
                    incidence_matrix[row_idx_bus2, line_idx * 3 + int(phase) - 1] = -1
        return incidence_matrix.T

    @staticmethod
    def prepare_incidence_df(incidence_matrix, buses, phases, line_data):
        incidence_df = pd.DataFrame(incidence_matrix, columns=[f"{bus}.{phase}" for bus in buses for phase in phases])
        incidence_df.index = [f"{line_name}.{phase}" for line_name in line_data.keys() for phase in phases]
        reference_bus = buses[0]
        reference_columns = [f"{reference_bus}.{phase}" for phase in phases]
        incidence_df.drop(columns=reference_columns, inplace=True)
        return incidence_df

    @staticmethod
    def compute_pseudo_inverse(incidence_df):
        A = incidence_df.to_numpy()
        F = np.linalg.pinv(A)
        return A, F

    def load_data_from_files(self):
        block_diagonal_matrix_hp = np.load(f'{self.path}/results/{self.test_case}Bus_block_diagonal_matrix_hp.npy')
        block_diagonal_matrix_hq = np.load(f'{self.path}/results/{self.test_case}Bus_block_diagonal_matrix_hq.npy')
        with open(f'{self.path}/results/{self.test_case}Bus_P_LDF_uniform_pu.json', 'r') as f:
            P_data = json.load(f)
        with open(f'{self.path}/results/{self.test_case}Bus_Q_LDF_uniform_pu.json', 'r') as f:
            Q_data = json.load(f)
        return block_diagonal_matrix_hp, block_diagonal_matrix_hq, P_data, Q_data

    @staticmethod
    def LinDist3Flow(H_p, H_q, gamma, rho, varrho, ref_bus, n_buses_3ph, n_branches, F, A, P_inj, Q_inj, V_AC, scenario_idx):
        """Compute the 3-phase LinDistFlow and sensitivities"""
        H_p = H_p.reshape(3 * n_branches, 3 * n_branches)
        H_q = H_q.reshape(3 * n_branches, 3 * n_branches)

        gamma = gamma.reshape(n_buses_3ph, 1)
        rho = rho.reshape(n_buses_3ph, 1)
        varrho = varrho.reshape(n_buses_3ph, 1)
        #Ref_V = np.array([1.056**2, 1.0374**2, 1.056**2])  # IEEE 13 BUS
        Ref_V = np.array([0.9890**2, 1.0245**2, 1.0146**2])  # IEEE 37 Bus

        Ref_V_expanded = np.tile(Ref_V, n_buses_3ph // 3).reshape(-1, 1)
        v = Ref_V_expanded - F @ H_p @ F.T @ (-P_inj + rho) - F @ H_q @ F.T @ (-Q_inj + varrho) + gamma

        P = A.T @ A
        B = np.zeros((len(A), 1))
        for i in range(len(A)):
            if np.any(P[i, :] != 0):
                B[i] = 1
        v = np.multiply(B, v)
        v_AC = np.array(V_AC[f"scenario_{scenario_idx}"])
        v_AC = v_AC[3:]  # delete the reference bus
        v_AC = np.array(v_AC).reshape((n_buses_3ph, 1))
        v = np.reshape(v, (n_buses_3ph, 1))

        return v, v_AC

    @staticmethod
    def initialize_voltage_matrix(v, bus_index, A, reference_voltages):
        V_matrix = np.ones((len(A) + 3, 1))
        ref_bus_idx = 0
        V_matrix[ref_bus_idx * 3:(ref_bus_idx + 1) * 3] = np.array(reference_voltages).reshape(-1, 1)
        start_idx = 0
        for bus, idx in bus_index.items():
            if idx > 0:
                V_matrix[idx * 3:(idx + 1) * 3] = v[start_idx:start_idx + 3].reshape(-1, 1)
                start_idx += 3
        return V_matrix

    def objective_function(self, H_p, H_q, rho, varrho, gamma, scen_start, scen_final):
        V_LDF = []
        V_DF = []
        n_buses_3ph = self.A.shape[0]
        n_branches = n_buses_3ph // 3

        P_pu = np.array([values[scenario_idx] for values in self.P_data.values()])
        Q_pu = np.array([values[scenario_idx] for values in self.Q_data.values()])

        P_flat_pu = P_pu.flatten().reshape(-1, 1)
        Q_flat_pu = Q_pu.flatten().reshape(-1, 1)

        v_results, v_AC_results = zip(*Parallel(n_jobs=-1)(delayed(self.LinDist3Flow)(
            H_p, H_q, gamma, rho, varrho, 0, n_buses_3ph, n_branches, self.F, self.A, P_flat_pu, Q_flat_pu, self.V_AC, s) for s in range(scen_start - 1, scen_final - 1)))

        V_LDF = np.concatenate(v_results)
        V_DF = np.concatenate(v_AC_results)

        Batch_size = scen_final - scen_start

        V_LDF = V_LDF ** 0.5

        objective = (1 / Batch_size) * (1 / (n_buses_3ph)) * np.linalg.norm(V_LDF - V_DF, 1)
        inf_norm_error = np.linalg.norm(V_LDF - V_DF, np.inf)

        # objective = (1/(n_buses_3ph)) *(1/Batch_size)* np.sum((V_LDF-V_DF**2)**2)

        return objective, inf_norm_error, V_LDF, V_DF

    def run_optimization(self):
        with open(f"results/{self.test_case}Bus_uniform_voltages.json", "r") as file:
            self.V_AC = json.load(file)

        """Initial parameters"""
        H_p      = block_diagonal_matrix_hp
        H_q      = block_diagonal_matrix_hq
        rho      = np.zeros(A.shape[0])
        varrho   = np.zeros(A.shape[0])
        gamma    = np.zeros(A.shape[0])
    
        """Optimized parameters"""
        #H_p = np.loadtxt(f"{self.path}/parameters/H_p_{self.test_case}bus_{self.optimization_method}.txt")
        #H_q = np.loadtxt(f"{self.path}/parameters/H_q_{self.test_case}bus_{self.optimization_method}.txt")
        #rho = np.loadtxt(f"{self.path}/parameters/gamma_P_{self.test_case}bus_{self.optimization_method}.txt")
        #varrho = np.loadtxt(f"{self.path}/parameters/gamma_Q_{self.test_case}bus_{self.optimization_method}.txt")
        #gamma = np.loadtxt(f"{self.path}/parameters/bias_{self.test_case}bus_{self.optimization_method}.txt")

        H_p = H_p.flatten()
        H_q = H_q.flatten()
        rho = np.reshape(rho, (self.A.shape[0],))
        varrho = np.reshape(varrho, (self.A.shape[0],))
        gamma = np.reshape(gamma, (self.A.shape[0],))

        start_time = time.process_time()

        objective, inf_norm_error, V_LDF, V_DF = self.objective_function(H_p, H_q, rho, varrho, gamma, self.scen_start, self.scen_final)
        print(f'Avg error {self.test_case} Bus: ', objective)
        print(f'Max  Error: {inf_norm_error}')

        end_time = time.process_time()
        execution_time = end_time - start_time
        print(f'Total execution time: {execution_time} seconds')


if __name__ == "__main__":
    optimizer = OLDFTesting()
    optimizer.run_optimization()


# Read High load scenarios an makes them ready for testing high load LDF

In [ ]:
import numpy as np
from opendssdirect import dss
import os
import json
import logging

class LoadScenarioGeneration:
    def __init__(self, dss_file, output_dir="results", S_base=500.0):
        self.dss_file = dss_file
        self.output_dir = output_dir
        self.S_base = S_base  # System base power in kVA (500 kVA)
        self.load_data = []
        self.bus_index = {}
        self.reference_bus = None
        self.P_scenarios = {}
        self.Q_scenarios = {}
        os.makedirs(self.output_dir, exist_ok=True)
        logging.basicConfig(filename=os.path.join(self.output_dir, 'log.txt'), level=logging.INFO)

    def load_system(self):
        try:
            dss.run_command(f'Redirect "{self.dss_file}"')
            dss.Solution.Solve()
            logging.info(f'Successfully loaded and solved DSS file: {self.dss_file}')
        except Exception as e:
            logging.error(f'Error loading DSS file: {e}')
            raise

    def load_line_data(self, line_data_file):
        try:
            with open(line_data_file, "r") as f:
                self.line_data = json.load(f)
            logging.info(f'Successfully loaded line data from {line_data_file}')
        except Exception as e:
            logging.error(f'Error loading line data: {e}')
            raise

    def generate_bus_index_from_lines(self):
        try:
            buses = []  # Use a list to maintain the order
            seen_buses = set()  # Helper set to avoid duplicates while maintaining order
            for line, data in self.line_data.items():
                bus1 = data['FromBus']
                bus2 = data['ToBus']
                if bus1 not in seen_buses:
                    buses.append(bus1)
                    seen_buses.add(bus1)
                if bus2 not in seen_buses:
                    buses.append(bus2)
                    seen_buses.add(bus2)

            self.bus_index = {bus: i for i, bus in enumerate(buses)}

            # Identify reference bus (typically the first one unless specified otherwise)
            self.reference_bus = buses[0] if buses else None
            logging.info('Successfully generated bus index from line data')
        except Exception as e:
            logging.error(f'Error generating bus index from line data: {e}')
            raise

    def load_scenarios(self, test_case):
        try:
            self.P_scenarios = np.load(os.path.join(self.output_dir, f"P_high_load_{test_case}.npy"), allow_pickle=True).item()
            self.Q_scenarios = np.load(os.path.join(self.output_dir, f"Q_high_load_{test_case}.npy"), allow_pickle=True).item()

            # Debug prints
            #print("Loaded P_scenarios:")
            #for load, scenarios in self.P_scenarios.items():
                #print(f"{load}: {scenarios}")

            #print("\nLoaded Q_scenarios:")
            #for load, scenarios in self.Q_scenarios.items():
                #print(f"{load}: {scenarios}")

            logging.info(f'Successfully loaded scenarios for {test_case}')
        except Exception as e:
            logging.error(f'Error loading scenarios: {e}')
            raise

    def extract_load_data(self):
        try:
            loads = dss.Loads.AllNames()
            for load in loads:
                dss.Loads.Name(load)
                bus = dss.CktElement.BusNames()[0].split('.')[0]  # Get only the base bus name
                phases = dss.Loads.Phases()
                conn = 'Delta' if dss.Loads.IsDelta() else 'Wye'
                kV = dss.Loads.kV()

                P_scenarios = self.P_scenarios.get(load, [])
                Q_scenarios = self.Q_scenarios.get(load, [])

                for scenario_idx in range(len(P_scenarios)):
                    P = P_scenarios[scenario_idx]
                    Q = Q_scenarios[scenario_idx]
                    #print(f"Load: {load}, Scenario: {scenario_idx}, P: {P}, Q: {Q}")

                    S = np.sqrt(P**2 + Q**2)  # Apparent power
                    power_factor = P / S if S != 0 else 1.0  # Power factor

                    if bus in self.bus_index:  # Ensure the bus is part of the indexed buses
                        self.load_data.append({
                            'name': load,
                            'bus': bus,
                            'phases': phases,
                            'connection': conn,
                            'P': P,
                            'Q': Q,
                            'kV': kV,
                            'power_factor': power_factor,
                            'bus_full': dss.CktElement.BusNames()[0],  # Full bus name with phases
                            'scenario': scenario_idx
                        })
            logging.info('Successfully extracted load data')
        except Exception as e:
            logging.error(f'Error extracting load data: {e}')
            raise

    def organize_power_data(self):
        try:
            scenario_count = len(next(iter(self.P_scenarios.values())))  # Assuming uniform scenario counts
            P_bus_phase = {bus: np.zeros((scenario_count, 3)) for bus in self.bus_index}
            Q_bus_phase = {bus: np.zeros((scenario_count, 3)) for bus in self.bus_index}

            for scenario_idx in range(scenario_count):
                for load in self.load_data:
                    if load['scenario'] == scenario_idx:
                        bus = load['bus']
                        connection = load['connection']
                        phase_indices = [int(phase) - 1 for phase in load['bus_full'].split('.')[1:] if phase.isdigit()]

                        P = load['P']
                        Q = load['Q']

                        if connection == 'Delta' and len(phase_indices) == 3:
                            # If three-phase Delta, distribute the load evenly across all three phases
                            for phase in phase_indices:
                                P_bus_phase[bus][scenario_idx][phase] = P / 3
                                Q_bus_phase[bus][scenario_idx][phase] = Q / 3
                        else:
                            # Otherwise, assign all the load to the first appearing phase
                            if phase_indices:
                                first_phase_index = phase_indices[0]
                                P_bus_phase[bus][scenario_idx][first_phase_index] = P
                                Q_bus_phase[bus][scenario_idx][first_phase_index] = Q

            logging.info('Successfully organized power data')
            return P_bus_phase, Q_bus_phase
        except Exception as e:
            logging.error(f'Error organizing power data: {e}')
            raise

    def convert_to_pu(self, P_bus_phase, Q_bus_phase):
        try:
            P_bus_phase_pu = {bus: P / self.S_base for bus, P in P_bus_phase.items()}
            Q_bus_phase_pu = {bus: Q / self.S_base for bus, Q in Q_bus_phase.items()}
            logging.info('Successfully converted power data to per unit values')
            return P_bus_phase_pu, Q_bus_phase_pu
        except Exception as e:
            logging.error(f'Error converting power data to per unit values: {e}')
            raise

    def remove_reference_bus_columns(self, P_bus_phase, Q_bus_phase):
        try:
            if self.reference_bus in P_bus_phase:
                del P_bus_phase[self.reference_bus]
            if self.reference_bus in Q_bus_phase:
                del Q_bus_phase[self.reference_bus]
            logging.info(f'Successfully removed reference bus {self.reference_bus} from power data')
        except Exception as e:
            logging.error(f'Error removing reference bus from power data: {e}')
            raise

    def save_power_data(self, P_bus_phase, Q_bus_phase, suffix="", test_case=""):
        try:
            P_bus_phase = {bus: P.tolist() for bus, P in P_bus_phase.items()}
            Q_bus_phase = {bus: Q.tolist() for bus, Q in Q_bus_phase.items()}

            with open(os.path.join(self.output_dir, f"{test_case}_P_LDF_High{suffix}.json"), "w") as fp:
                json.dump(P_bus_phase, fp, indent=4)
            with open(os.path.join(self.output_dir, f"{test_case}_Q_LDF_High{suffix}.json"), "w") as fq:
                json.dump(Q_bus_phase, fq, indent=4)
            logging.info(f'P and Q data saved to {self.output_dir} as {suffix}')
        except Exception as e:
            logging.error(f'Error saving power data: {e}')
            raise

    def save_load_data(self, test_case=""):
        try:
            with open(os.path.join(self.output_dir, f"{test_case}_load_data.json"), "w") as fl:
                json.dump(self.load_data, fl, indent=4)
            logging.info(f'Load data saved to {self.output_dir}')
        except Exception as e:
            logging.error(f'Error saving load data: {e}')
            raise

    def load_power_data(self, suffix="", test_case=""):
        try:
            with open(os.path.join(self.output_dir, f"{test_case}_P_LDF_High{suffix}.json"), "r") as fp:
                P_bus_phase = json.load(fp)
            with open(os.path.join(self.output_dir, f"{test_case}_Q_LDF_High{suffix}.json"), "r") as fq:
                Q_bus_phase = json.load(fq)
            logging.info(f'Successfully loaded power data with suffix {suffix}')
            return P_bus_phase, Q_bus_phase
        except Exception as e:
            logging.error(f'Error loading power data: {e}')
            raise

    def power_data_to_matrix(self, power_data, scenario_idx):
        power_matrix = np.zeros((len(self.bus_index) - 1, 3))  # Adjust for removed reference bus
        try:
            for bus, idx in self.bus_index.items():
                if bus != self.reference_bus and bus in power_data:
                    power_matrix[idx - (1 if idx > self.bus_index[self.reference_bus] else 0), :] = power_data[bus][scenario_idx]
            logging.info('Successfully converted power data to matrix')
            return power_matrix
        except Exception as e:
            logging.error(f'Error converting power data to matrix: {e}')
            raise

if __name__ == "__main__":
    try:
        TestCase = '13Bus'
        TestCase = '37Bus'

        #simulator = LoadScenarioGeneration(dss_file=f"/Users/babaktaheri/Desktop/OLDF/Multi-phase/data/IEEETestCases/{TestCase}/IEEE13Nodeckt.dss", S_base=500.0)
        simulator = LoadScenarioGeneration(dss_file=f"/Users/babaktaheri/Desktop/OLDF/Multi-phase/data/IEEETestCases/{TestCase}/ieee37.dss", S_base=500.0)

        simulator.load_system()
        simulator.load_line_data(line_data_file=f"results/{TestCase}_line_data.json")  # Update with correct file path
        simulator.generate_bus_index_from_lines()
        simulator.load_scenarios(TestCase)
        simulator.extract_load_data()
        P_bus_phase, Q_bus_phase = simulator.organize_power_data()
        
        # Remove reference bus columns
        simulator.remove_reference_bus_columns(P_bus_phase, Q_bus_phase)

        # Convert to per unit values
        P_bus_phase_pu, Q_bus_phase_pu = simulator.convert_to_pu(P_bus_phase, Q_bus_phase)
        
        simulator.save_power_data(P_bus_phase, Q_bus_phase, suffix="", test_case=TestCase)
        simulator.save_power_data(P_bus_phase_pu, Q_bus_phase_pu, suffix="_pu", test_case=TestCase)
        simulator.save_load_data(test_case=TestCase)

        P_bus_phase, Q_bus_phase = simulator.load_power_data(suffix="", test_case=TestCase)
        P_bus_phase_pu, Q_bus_phase_pu = simulator.load_power_data(suffix="_pu", test_case=TestCase)

        # Example: Displaying the power data matrix for the first scenario (index 0)
        scenario_idx = 0
        P_matrix = simulator.power_data_to_matrix(P_bus_phase, scenario_idx)
        Q_matrix = simulator.power_data_to_matrix(Q_bus_phase, scenario_idx)
        P_matrix_pu = simulator.power_data_to_matrix(P_bus_phase_pu, scenario_idx)
        Q_matrix_pu = simulator.power_data_to_matrix(Q_bus_phase_pu, scenario_idx)

        #print(f"P matrix for scenario {scenario_idx} (in kW):\n", P_matrix)
        #print(f"Q matrix for scenario {scenario_idx} (in kVar):\n", Q_matrix)
        #print(f"P matrix for scenario {scenario_idx} (in pu):\n", P_matrix_pu)
        #print(f"Q matrix for scenario {scenario_idx} (in pu):\n", Q_matrix_pu)
        print("Bus index:\n", simulator.bus_index)
    except Exception as e:
        logging.error(f'Error in main execution: {e}')
        print(f'Error in main execution: {e}')


# High Load Testing

In [ ]:
import pandas as pd
import numpy as np
from opendssdirect import dss
import time
import os
import json
from scipy.optimize import minimize
from joblib import Parallel, delayed
import random

class VoltageProfileOptimizer:
    def __init__(self, test_case='37', optimization_method='TNC', path='/Users/babaktaheri/Desktop/OLDF/Multi-phase'):
        self.test_case = test_case
        self.optimization_method = optimization_method
        self.path = path
        self.dss_master_file_path = f"{self.path}/data/IEEETestCases/{self.test_case}Bus/ieee37.dss"
        #self.reference_voltages = np.array([1.056, 1.0374, 1.056]) #IEEE 13Bus
        self.reference_voltages = np.array([0.9890, 1.0245, 1.0146])  # IEEE 37 Bus
        self.load_system()
        self.line_data = self.load_line_data_from_json(f"{self.path}/results/{self.test_case}Bus_line_data.json")
        self.buses, self.phases, self.bus_index = self.extract_buses_and_phases(self.line_data)
        self.incidence_matrix = self.create_incidence_matrix(self.buses, self.phases, self.bus_index, self.line_data)
        self.incidence_df = self.prepare_incidence_df(self.incidence_matrix, self.buses, self.phases, self.line_data)
        self.A, self.F = self.compute_pseudo_inverse(self.incidence_df)
        self.block_diagonal_matrix_hp, self.block_diagonal_matrix_hq, self.P_data, self.Q_data = self.load_data_from_files()
        self.scen_start = 1
        self.scen_final = 31
        self.H_p = self.block_diagonal_matrix_hp
        self.H_q = self.block_diagonal_matrix_hq
        self.rho = np.zeros(self.A.shape[0])
        self.varrho = np.zeros(self.A.shape[0])
        self.gamma = np.zeros(self.A.shape[0])

    def load_system(self):
        dss.run_command("Clear")
        dss.run_command(f'Redirect "{self.dss_master_file_path}"')
        if dss.Circuit.Name() == "":
            raise Exception("No active circuit. Please check the DSS master file.")

    @staticmethod
    def load_line_data_from_json(file_path):
        with open(file_path, 'r') as file:
            line_data = json.load(file)
        return line_data

    @staticmethod
    def extract_buses_and_phases(line_data):
        buses = []
        phases = {'1', '2', '3'}
        seen_buses = set()
        for line_info in line_data.values():
            bus1_base = line_info['FromBus'].split('.')[0]
            bus2_base = line_info['ToBus'].split('.')[0]
            if bus1_base not in seen_buses:
                buses.append(bus1_base)
                seen_buses.add(bus1_base)
            if bus2_base not in seen_buses:
                buses.append(bus2_base)
                seen_buses.add(bus2_base)
        bus_index = {bus: i for i, bus in enumerate(buses)}
        return buses, phases, bus_index

    @staticmethod
    def create_incidence_matrix(buses, phases, bus_index, line_data):
        incidence_matrix = np.zeros((len(buses) * len(phases), len(line_data) * len(phases)), dtype=int)
        for line_idx, (line_id, line_info) in enumerate(line_data.items()):
            bus1 = line_info['FromBus']
            bus2 = line_info['ToBus']
            bus1_phases = line_info['Bus1Phases']
            bus2_phases = line_info['Bus2Phases']
            for phase in phases:
                if phase in bus1_phases:
                    row_idx_bus1 = bus_index[bus1] * 3 + int(phase) - 1
                    incidence_matrix[row_idx_bus1, line_idx * 3 + int(phase) - 1] = 1
                if phase in bus2_phases:
                    row_idx_bus2 = bus_index[bus2] * 3 + int(phase) - 1
                    incidence_matrix[row_idx_bus2, line_idx * 3 + int(phase) - 1] = -1
        return incidence_matrix.T

    @staticmethod
    def prepare_incidence_df(incidence_matrix, buses, phases, line_data):
        incidence_df = pd.DataFrame(incidence_matrix, columns=[f"{bus}.{phase}" for bus in buses for phase in phases])
        incidence_df.index = [f"{line_name}.{phase}" for line_name in line_data.keys() for phase in phases]
        reference_bus = buses[0]
        reference_columns = [f"{reference_bus}.{phase}" for phase in phases]
        incidence_df.drop(columns=reference_columns, inplace=True)
        return incidence_df

    @staticmethod
    def compute_pseudo_inverse(incidence_df):
        A = incidence_df.to_numpy()
        F = np.linalg.pinv(A)
        return A, F

    def load_data_from_files(self):
        block_diagonal_matrix_hp = np.load(f'{self.path}/results/{self.test_case}Bus_block_diagonal_matrix_hp.npy')
        block_diagonal_matrix_hq = np.load(f'{self.path}/results/{self.test_case}Bus_block_diagonal_matrix_hq.npy')
        with open(f'{self.path}/results/{self.test_case}Bus_P_LDF_High_pu.json', 'r') as f:
            P_data = json.load(f)
        with open(f'{self.path}/results/{self.test_case}Bus_Q_LDF_High_pu.json', 'r') as f:
            Q_data = json.load(f)
        return block_diagonal_matrix_hp, block_diagonal_matrix_hq, P_data, Q_data

    @staticmethod
    def LinDist3Flow(H_p, H_q, gamma, rho, varrho, ref_bus, n_buses_3ph, n_branches, F, A, P_inj, Q_inj, V_AC, scenario_idx):
        """Compute the 3-phase LinDistFlow and sensitivities"""
        H_p = H_p.reshape(3 * n_branches, 3 * n_branches)
        H_q = H_q.reshape(3 * n_branches, 3 * n_branches)

        gamma = gamma.reshape(n_buses_3ph, 1)
        rho = rho.reshape(n_buses_3ph, 1)
        varrho = varrho.reshape(n_buses_3ph, 1)
        # Ref_V = np.array([1.056**2, 1.0374**2, 1.056**2]) #IEEE 13Bus
        Ref_V = np.array([0.9890**2, 1.0245**2, 1.0146**2])  # IEEE 37 Bus

        Ref_V_expanded = np.tile(Ref_V, n_buses_3ph // 3).reshape(-1, 1)
        v = Ref_V_expanded - F @ H_p @ F.T @ (-P_inj + rho) - F @ H_q @ F.T @ (-Q_inj + varrho) + gamma

        P = A.T @ A
        B = np.zeros((len(A), 1))
        for i in range(len(A)):
            if np.any(P[i, :] != 0):
                B[i] = 1
        v = np.multiply(B, v)
        v_AC = np.array(V_AC[f"scenario_{scenario_idx}"])
        v_AC = v_AC[3:]  # delete the reference bus
        v_AC = np.array(v_AC).reshape((n_buses_3ph, 1))
        v = np.reshape(v, (n_buses_3ph, 1))

        return v, v_AC

    @staticmethod
    def initialize_voltage_matrix(v, bus_index, A, reference_voltages):
        V_matrix = np.ones((len(A) + 3, 1))
        ref_bus_idx = 0
        V_matrix[ref_bus_idx * 3:(ref_bus_idx + 1) * 3] = np.array(reference_voltages).reshape(-1, 1)
        start_idx = 0
        for bus, idx in bus_index.items():
            if idx > 0:
                V_matrix[idx * 3:(idx + 1) * 3] = v[start_idx:start_idx + 3].reshape(-1, 1)
                start_idx += 3
        return V_matrix

    def objective_function(self, H_p, H_q, rho, varrho, gamma, scen_start, scen_final):
        V_LDF = []
        V_DF = []
        n_buses_3ph = self.A.shape[0]
        n_branches = n_buses_3ph // 3

        P_pu = np.array([values[scenario_idx] for values in self.P_data.values()])
        Q_pu = np.array([values[scenario_idx] for values in self.Q_data.values()])

        P_flat_pu = P_pu.flatten().reshape(-1, 1)
        Q_flat_pu = Q_pu.flatten().reshape(-1, 1)

        v_results, v_AC_results = zip(*Parallel(n_jobs=-1)(delayed(self.LinDist3Flow)(
            H_p, H_q, gamma, rho, varrho, 0, n_buses_3ph, n_branches, self.F, self.A, P_flat_pu, Q_flat_pu, self.V_AC, s) for s in range(scen_start - 1, scen_final - 1)))

        V_LDF = np.concatenate(v_results)
        V_DF = np.concatenate(v_AC_results)

        Batch_size = scen_final - scen_start

        V_LDF = V_LDF ** 0.5

        objective = (1 / Batch_size) * (1 / (n_buses_3ph)) * np.linalg.norm(V_LDF - V_DF, 1)
        inf_norm_error = np.linalg.norm(V_LDF - V_DF, np.inf)

        # objective = (1/(n_buses_3ph)) *(1/Batch_size)* np.sum((V_LDF-V_DF**2)**2)

        return objective, inf_norm_error, V_LDF, V_DF

    def run_optimization(self):
        with open(f"results/{self.test_case}Bus_High_voltages.json", "r") as file:
            self.V_AC = json.load(file)

        """Initial parameters"""
        H_p      = block_diagonal_matrix_hp
        H_q      = block_diagonal_matrix_hq
        rho      = np.zeros(A.shape[0])
        varrho   = np.zeros(A.shape[0])
        gamma    = np.zeros(A.shape[0])    
            
        #H_p = np.loadtxt(f"{self.path}/parameters/H_p_{self.test_case}bus_{self.optimization_method}.txt")
        #H_q = np.loadtxt(f"{self.path}/parameters/H_q_{self.test_case}bus_{self.optimization_method}.txt")
        #rho = np.loadtxt(f"{self.path}/parameters/gamma_P_{self.test_case}bus_{self.optimization_method}.txt")
        #varrho = np.loadtxt(f"{self.path}/parameters/gamma_Q_{self.test_case}bus_{self.optimization_method}.txt")
        #gamma = np.loadtxt(f"{self.path}/parameters/bias_{self.test_case}bus_{self.optimization_method}.txt")

        H_p = H_p.flatten()
        H_q = H_q.flatten()
        rho = np.reshape(rho, (self.A.shape[0],))
        varrho = np.reshape(varrho, (self.A.shape[0],))
        gamma = np.reshape(gamma, (self.A.shape[0],))

        start_time = time.process_time()

        objective, inf_norm_error, V_LDF, V_DF = self.objective_function(H_p, H_q, rho, varrho, gamma, self.scen_start, self.scen_final)
        print(f'Avg error {self.test_case} Bus: ', objective)
        print(f'Max  Error: {inf_norm_error}')

        end_time = time.process_time()
        execution_time = end_time - start_time
        print(f'Total execution time: {execution_time} seconds')


if __name__ == "__main__":
    optimizer = VoltageProfileOptimizer()
    optimizer.run_optimization()